![](imgs/logo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.


## Spark SQL - statystyki z użyciem SQL 

W katalogu `data/` znajduje się plik `rollingsales_bronx.json`.

Poniżej analizujemy dane statystyczne domów: liczność, średnie powierzchnie, lata i ceny, grupując po dzielnicy i typie.

In [1]:
import pyspark
import pyspark.sql.functions as func
sc = pyspark.SparkContext(appName='HouseStatsSQL')
sqlContext = pyspark.sql.SQLContext(sc)

In [6]:
# Czytanie danych
sales = sqlContext.read.json('data/rollingsales_bronx.json')
# Nie zapomnij zarejestrować tabeli
sales.registerTempTable('sales')
sales.printSchema()
sales.show()

root
 |-- grossArea: long (nullable = true)
 |-- hood: string (nullable = true)
 |-- landArea: long (nullable = true)
 |-- price: long (nullable = true)
 |-- type: string (nullable = true)
 |-- year: long (nullable = true)

+---------+--------------------+--------+------+--------------------+----+
|grossArea|                hood|landArea| price|                type|year|
+---------+--------------------+--------+------+--------------------+----+
|     2048|BATHGATE         ...|    1842|355000|01  ONE FAMILY HO...|1901|
|     1290|BATHGATE         ...|    1103|474819|01  ONE FAMILY HO...|1910|
|     1344|BATHGATE         ...|    1986|210000|01  ONE FAMILY HO...|1899|
|     1431|BATHGATE         ...|    2329|343116|01  ONE FAMILY HO...|1901|
|     4452|BATHGATE         ...|    1855|     0|02  TWO FAMILY HO...|1931|
|     2400|BATHGATE         ...|    2000|316500|02  TWO FAMILY HO...|1993|
|     2394|BATHGATE         ...|    2498|390000|02  TWO FAMILY HO...|1995|
|     1542|BATHGATE       

In [7]:
query = """
SELECT 
    hood,
    type,
    COUNT(*) as count,
    AVG(landArea) as avgLandArea,
    AVG(year) as avgYear,
    AVG(price) as avgPrice
FROM sales
GROUP BY hood, type
ORDER BY hood, type
"""
a1 = sqlContext.sql(query)
a1.show()

+--------------------+--------------------+-----+------------------+------------------+------------------+
|                hood|                type|count|       avgLandArea|           avgYear|          avgPrice|
+--------------------+--------------------+-----+------------------+------------------+------------------+
|BATHGATE         ...|01  ONE FAMILY HO...|    4|            1815.0|           1902.75|         345733.75|
|BATHGATE         ...|02  TWO FAMILY HO...|   10|            2131.1|            1947.2|          203427.6|
|BATHGATE         ...|03  THREE FAMILY ...|    6|            2252.5|            1919.0| 292019.1666666667|
|BATHGATE         ...|05  TAX CLASS 1 V...|    1|            2099.0|               0.0|           40730.0|
|BATHGATE         ...|07  RENTALS - WAL...|    6|3161.6666666666665|            1924.5| 424286.1666666667|
|BATHGATE         ...|10  COOPS - ELEVA...|    1|               0.0|               0.0|               4.0|
|BATHGATE         ...|14  RENTALS - 4

## Zadania

* Popraw wyniki usuwając błędne dane.
* Zastosuj odpowiednią prezentację dany w zależności od typu.
* Policz średnie dla domów z XX w. tylko dla grup zawierających więcej niż 10 domów.
* ★ Policz średnie tylko dla 10 najbogatszych dzielnic.

In [19]:
query = """
SELECT 
    hood,
    type,
    COUNT(*) as count,
    CAST(AVG(landArea) AS DECIMAL(36,2)) as avgLandArea,
    CAST(AVG(year) AS INT) as avgYear,
    CAST(AVG(price) AS DECIMAL(36,2)) as avgPrice
FROM sales
WHERE 
    year > 1900 AND year < 2001 AND landArea <> 0 AND price <> 0
GROUP BY hood, type
HAVING (count > 10)
ORDER BY hood, type
"""
a1 = sqlContext.sql(query)
a1.show()

+--------------------+--------------------+-----+-----------+-------+----------+
|                hood|                type|count|avgLandArea|avgYear|  avgPrice|
+--------------------+--------------------+-----+-----------+-------+----------+
|BAYCHESTER       ...|01  ONE FAMILY HO...|   76|    2852.88|   1940| 237717.00|
|BAYCHESTER       ...|02  TWO FAMILY HO...|   67|    2795.90|   1948| 332438.84|
|BAYCHESTER       ...|03  THREE FAMILY ...|   11|    2572.27|   1958| 408090.91|
|BEDFORD PARK/NORW...|01  ONE FAMILY HO...|   11|    2839.09|   1909| 541681.82|
|BEDFORD PARK/NORW...|02  TWO FAMILY HO...|   21|    2443.67|   1914| 354427.71|
|BEDFORD PARK/NORW...|07  RENTALS - WAL...|   21|    8995.29|   1922|3907523.81|
|BEDFORD PARK/NORW...|08  RENTALS - ELE...|   13|   10067.77|   1931|3668538.46|
|BELMONT          ...|02  TWO FAMILY HO...|   18|    2724.94|   1915| 296362.00|
|BRONXDALE        ...|01  ONE FAMILY HO...|   37|    2143.81|   1945| 239997.59|
|BRONXDALE        ...|02  TW

In [24]:
query = """ 
SELECT 
    hood,
    CAST(AVG(price) AS DECIMAL(36,2)) as avgPrice
FROM sales
WHERE 
    year > 1900 AND year < 2001 AND landArea <> 0 AND price <> 0
GROUP BY hood
ORDER BY avgPrice desc
LIMIT 10
"""
a1 = sqlContext.sql(query)
a1.show()

+--------------------+----------+
|                hood|  avgPrice|
+--------------------+----------+
|HIGHBRIDGE/MORRIS...|4667409.98|
|MOUNT HOPE/MOUNT ...|3506842.44|
|MOTT HAVEN/PORT M...|3322260.41|
|PELHAM PARKWAY SO...|2442798.89|
|CO-OP CITY       ...|2286666.67|
|FORDHAM          ...|2062679.20|
|BEDFORD PARK/NORW...|1879474.22|
|MELROSE/CONCOURSE...|1730579.11|
|KINGSBRIDGE/JEROM...|1569162.40|
|FIELDSTON        ...|1538845.38|
+--------------------+----------+

